# EGM

**Table of contents**<a id='toc0_'></a>    
- 1. [Imports](#toc1_)    
- 2. [Model](#toc2_)    
- 3. [Numba](#toc3_)    
  - 3.1. [Compile](#toc3_1_)    
  - 3.2. [Euler-error](#toc3_2_)    
  - 3.3. [Transfer func](#toc3_3_)    
  - 3.4. [Multiple Rs and transfer funcs](#toc3_4_)    
  - 3.5. [Save](#toc3_5_)    
  - 3.6. [Time](#toc3_6_)    
- 4. [C++](#toc4_)    
  - 4.1. [Compile](#toc4_1_)    
  - 4.2. [Time](#toc4_2_)    
  - 4.3. [No threading](#toc4_3_)    
  - 4.4. [R](#toc4_4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Imports](#toc0_)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import torch
import time
from copy import deepcopy
import pickle
import matplotlib.pyplot as plt
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

In [ ]:
from BufferStockModel import BufferStockModelClass
from BufferStockModelEGM import BufferStockModelEGMClass
from EconDLSolvers import choose_gpu

## 2. <a id='toc2_'></a>[Approximate Model](#toc0_)

In [ ]:
par_dict = {'use_reg':True}

# model_reg = BufferStockModelEGMClass(par =par_dict, egm = egm_dict)
model_approx = BufferStockModelEGMClass(par =par_dict)

In [ ]:
t0 = time.perf_counter()
model_approx.link_to_cpp(do_print=False)
print(f'Compile: {time.perf_counter()-t0:.1f} seconds')
t0 = time.perf_counter()
model_approx.cpp.solve_all(model_approx.par,model_approx.egm)
EGM_time = time.perf_counter()-t0
model_approx.simulate_R() # same states and shocks as in .sim
model_approx.cpp.compute_euler_errors(model_approx.par,model_approx.egm,model_approx.sim)
model_approx.info['time'] = EGM_time

model_approx_dict = model_approx.as_dict()
model_save = deepcopy(model_approx_dict) # save the model to a file
filename = 'GUV_DP_reg.pkl'
with open(f'{filename}', 'wb') as f:
    pickle.dump(model_save, f)	



model_approx.cpp.delink()

## 3. <a id='toc2_'></a>[Baseline Model](#toc0_)

In [ ]:
model_base = BufferStockModelEGMClass()

In [ ]:
t0 = time.perf_counter()
model_base.link_to_cpp(do_print=False)
print(f'Compile: {time.perf_counter()-t0:.1f} seconds')
t0 = time.perf_counter()
model_base.cpp.solve_all(model_base.par,model_base.egm)
EGM_time = time.perf_counter()-t0
model_base.simulate_R() # same states and shocks as in .sim
model_base.cpp.compute_euler_errors(model_base.par,model_base.egm,model_base.sim)
model_base.info['time'] = EGM_time

model_base_dict = model_base.as_dict()
model_save = deepcopy(model_base_dict) # save the model to a file
model_save['egm'].sol_con = None

filename = 'GUV_DP_base.pkl'
with open(f'{filename}', 'wb') as f:
    pickle.dump(model_save, f)	



model_base.cpp.delink()